# Instalação das Bibliotecas Necessárias

In [ ]:
!pip install PyDrive google-api-python-client google-auth google-auth-oauthlib google-cloud-storage requests pyjwt

# Realização de importações necessárias

In [ ]:
import pandas as pd
import unicodedata
import requests
import json
import io
import os
import re

from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaIoBaseDownload
from google.cloud.exceptions import NotFound
from googleapiclient.discovery import build
from google.cloud import storage, bigquery
from google.colab import files, auth
from pandas_gbq import to_gbq
from datetime import date

# Recebimento de arquivos via Google Drive e envio para Google Cloud Storage (Bucket)

In [ ]:
SERVICE_ACCOUNT_FILE = "" # Insira as credenciais da Conta de Serviço
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/documents",
    "https://www.googleapis.com/auth/drive"
]
FOLDER_ID = "" # Insira o ID da pasta que contém os Dados
BUCKET_NAME = "" # Insira o nome da Bucket no Google Cloud Storage

# Autenticação para o Google Drive e Google Cloud Storage
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build("drive", "v3", credentials=credentials)
storage_client = storage.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

# Listar apenas os arquivos da pasta específica
print("\nListando arquivos na pasta específica:")
query = f"'{FOLDER_ID}' in parents and trashed=false"
results = drive_service.files().list(
    q=query,
    pageSize=100,  # Ajuste conforme necessário
    orderBy="modifiedTime desc",
    fields="files(id, name, modifiedTime)"
).execute()
files = results.get("files", [])

if not files:
    print("❌ Nenhum arquivo encontrado na pasta especificada.")
else:
    print(f"Total de arquivos encontrados: {len(files)}")

    # Diretório local para salvar arquivos
    local_folder = "downloads_do_drive"
    os.makedirs(local_folder, exist_ok=True)

    for file in files:
        file_id = file["id"]
        file_name = file["name"]
        print(f"\n🌐Baixando arquivo: Nome: {file_name}, ID: {file_id}")

        # Baixar o arquivo do Google Drive
        request = drive_service.files().get_media(fileId=file_id)
        file_data = io.BytesIO()
        downloader = MediaIoBaseDownload(file_data, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}% ✅ concluído para o arquivo '{file_name}'.")
        file_data.seek(0)

        # Salvar o arquivo localmente
        local_path = os.path.join(local_folder, file_name)
        with open(local_path, "wb") as local_file:
            local_file.write(file_data.read())
        print(f"Arquivo '{file_name}' salvo localmente em '{local_path}'.")

        # Resetar o ponteiro do stream para o início antes do upload
        file_data.seek(0)

        # Enviar o arquivo para o bucket
        pasta_bucket_envio = f"dados_brutos/{file_name}"
        print(f"\nFazendo upload do arquivo '{file_name}' para o caminho {pasta_bucket_envio} no bucket '{BUCKET_NAME}'...")
        bucket = storage_client.bucket(BUCKET_NAME)
        blob = bucket.blob(pasta_bucket_envio)
        blob.upload_from_file(file_data, content_type='application/octet-stream')  # Ajuste o content_type se necessário
        print(f"✅ Arquivo '{file_name}' enviado com sucesso para o caminho '{pasta_bucket_envio}' no bucket '{BUCKET_NAME}'.")

In [ ]:
# Configurar o ID da pasta de destino no Google Drive
pasta_drive_backup = "" # Insira o ID da pasta de destino

# Função para mover os arquivos baixados para a pasta de destino no Google Drive
def mover_arquivos_para_pasta_destino(files, pasta_drive_backup):
    for file in files:
        file_id = file["id"]
        file_name = file["name"]
        print(f"\nMovendo arquivo: Nome: {file_name}, ID: {file_id}")

        previous_parents = ",".join([FOLDER_ID])  # Usa o ID da pasta de origem já definido
        drive_service.files().update(
            fileId=file_id,
            addParents=pasta_drive_backup,
            removeParents=previous_parents,
            fields="id, parents"
        ).execute()

        print(f"Arquivo '{file_name}' movido para a pasta de destino com ID '{pasta_drive_backup}'.")

# Chamar a função para mover os arquivos para a pasta de destino
mover_arquivos_para_pasta_destino(files, pasta_drive_backup)


# Excluir os arquivos locais após o envio

In [ ]:
for file_name in os.listdir(local_folder):  # Lista todos os arquivos na pasta local
    local_path = os.path.join(local_folder, file_name)  # Caminho completo do arquivo
    if os.path.isfile(local_path):  # Verifica se é um arquivo (e não um diretório)
        os.remove(local_path)  # Exclui o arquivo
        print(f"Arquivo local '{file_name}' excluído.")
    else:
        print(f"'{file_name}' não é um arquivo, ignorado.")

# Funções para Normalização e Envio de DataSets

In [ ]:
# TRATA OS ARQUIVOS (.sql)

def sql_para_dataframe(blob):

    # Converte um arquivo SQL contendo apenas comandos INSERT INTO em um DataFrame.
    with blob.open("r", encoding="utf-8") as arquivo:
         conteudo_sql = arquivo.read()

    # Ajuste na regex para capturar corretamente o nome da tabela e os valores
    padrao_insert = re.compile(r"INSERT INTO `?([\w\s&]+)`?\s*\((.*?)\)\s*VALUES\s*(.+);", re.IGNORECASE)

    correspondencias = padrao_insert.findall(conteudo_sql)
    if not correspondencias:
        print(f"⚠️ Nenhum INSERT INTO encontrado em {blob.name}")
        return pd.DataFrame()

    dados = []
    colunas = []

    for correspondencia in correspondencias:
        nome_tabela, parte_colunas, parte_valores = correspondencia

        # Remover espaços extras e formatar colunas
        if not colunas:
            colunas = [col.strip().strip("`") for col in parte_colunas.split(",")]

        # Ajuste na separação dos valores para evitar que quebre errado por conta de vírgulas internas
        valores = re.findall(r"\((.*?)\)", parte_valores)

        # Manter estrutura correta dos valores (respeitando strings com vírgulas entre aspas)
        for valor in valores:
            valores_processados = [v.strip().strip("'") for v in re.split(r",(?=(?:[^']*'[^']*')*[^']*$)", valor)]
            dados.append(valores_processados)

    dataframe = pd.DataFrame(dados, columns=colunas)

    if dataframe.empty:
        print(f"⚠️ Nenhum dado extraído corretamente de {blob.name}")
    else:
        print(f"✅ Dados extraídos de {blob.name} corretamente!")

    return dataframe

In [ ]:
"""
Procura colunas únicas dentro do arquivo para gerar um nome padrão,
Faz o upload do novo arquivo para a bucket de destino e
Exclui o arquivo original
"""
def renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder):
        client = storage.Client()
        source_bucket = client.bucket(source_bucket_name)
        dest_bucket = client.bucket(dest_bucket_name)
        # Verifica se a coluna unica está presente
        if "nivel" in df.columns and blob.name.endswith(".json"):
            print(f"Arquivo: {blob.name} é ativos pj")
            new_file_name = "ativos_pj"
        elif "nivel" in df.columns and blob.name.endswith(".xlsx"):
            print(f"Arquivo: {blob.name} é ativos clt")
            new_file_name = "ativos_clt"
        elif "abertura_vaga" in df.columns:
            print(f"Arquivo: {blob.name} é abertura de  vagas")
            new_file_name = "vagas_rs"
        elif "Data de conclusão" in df.columns:
            print(f"Arquivo: {blob.name} é censo de diversidade")
            new_file_name = "censo_diversidade"
        elif "tipo_desligamento" in df.columns:
            print(f"Arquivo: {blob.name} é relação  de desligados")
            new_file_name = "relacao_desligados"
        elif "duracao" in df.columns:
            print(f"Arquivo: {blob.name} é Ferias Clt")
            new_file_name = "ferias_clt"
        elif "abertura vaga" in df.columns:
            print(f"Arquivo: {blob.name} é Vagas R&S")
            new_file_name = "vagas_rs"
        elif "Quant. dias" in df.columns:
            print(f"Arquivo: {blob.name} é férias pj")
            new_file_name = "ferias_pj"
        elif "status" in df.columns and blob.name.endswith(".sql"):
            print(f"❌Arquivo: {blob.name} é Vagas R&S.sql")
            new_file_name = "vagas_rs"
        else:
            print(f"❌ Arquivo: {blob.name} não contem coluna unica")
            # Deletar o arquivo origem
            try:
                blob.delete()
                print(f"🆗Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")
            return

        caminho = f"{dest_folder}/{new_file_name}.csv"

        # Salva o novo arquivo em memória
        print(f"Arquivo: {blob.name} sendo salvo na memoria")
        output = io.StringIO()
        df.to_csv(output, index=False)
        output.seek(0)

        # Faz o upload do novo arquivo para a bucket de destino
        print(f"⏫Arquivo: {blob.name} sendo upado para dados_pre_processados")
        new_blob = dest_bucket.blob(caminho)
        new_blob.upload_from_string(output.getvalue(), content_type="text/csv")

        # Verificação de sucesso do upload
        if new_blob.exists():
            print(f"✅ Novo arquivo criado com sucesso na pasta pre_preocessados: {new_file_name}")
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")
        else:
            print(f"❌ Falha ao criar o novo arquivo para {blob.name}. O arquivo original não foi excluído.")


In [ ]:
"""
Lista todos os arquivos de uma bucket de origem.
Tranforma o arquivo para csv baseado e sua extensão
Chama a função "renomeia" definida acima
"""

def processa_arquivos(source_bucket_name, source_folder, dest_bucket_name, dest_folder):
    # Cria cliente do Google Cloud Storage
    client = storage.Client()

    # Referencia as buckets de origem e destino
    source_bucket = client.bucket(source_bucket_name)
    dest_bucket = client.bucket(dest_bucket_name)

    # Lista todos os blobs (arquivos) na bucket de origem
    blobs = source_bucket.list_blobs(prefix=source_folder)

    # Trata cada arquivo conforme sua extensão
    for blob in blobs:
        if  blob.name.endswith(".json"):
            ''' Lê o formato jason, transforma em csv  e renomeia'''
            print(f"Arquivo: {blob.name} sendo precessado")
            file_data = blob.download_as_bytes()
            data = json.loads(file_data.decode('utf-8'))

            try:
                # Tenta carregar o arquivo em um DataFrame
                df = pd.DataFrame(data)
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
            # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
                try:
                    blob.delete()
                    print(f"🆗Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
                except Exception as e:
                    print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")

        elif blob.name.endswith(".txt"):
            #Lê o formato  texto, transforma em csv  e renomeia
            print(f"Arquivo: {blob.name} sendo precessado")
            file_data = blob.download_as_bytes()
            try:
            # Tenta carregar o arquivo em um DataFrame
                df = pd.read_csv(io.StringIO(file_data.decode('latin-1')), sep='\t')
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
            # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")

        elif blob.name.endswith(".xlsx"):
            # Lê o formato  excel, transforma em csv  e renomeia
            print(f"Arquivo: {blob.name} sendo precessado")
            file_data = blob.download_as_bytes()

            try:
                # Tenta carregar o arquivo em um DataFrame
                df = pd.read_excel(io.BytesIO(file_data))
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
                # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
                try:
                    blob.delete()
                    print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
                except Exception as e:
                    print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")

        elif  blob.name.endswith(".xls"):
            # Lê o formato  excel, transforma em csv  e renomeia
            print(f"Arquivo: {blob.name} sendo precessado")
            file_data = blob.download_as_bytes()

            try:
                # Tenta carregar o arquivo em um DataFrame
                df = pd.read_excel(io.BytesIO(file_data))
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
                # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")

        elif blob.name.endswith(".gsheet"):
            # Lê o formato  google sheets, transforma em csv  e renomeia
            print(f"Arquivo: {blob.name} sendo precessado")

            try:
                # Tenta carregar o arquivo em um DataFrame
                df = pd.read_excel(io.BytesIO(file_data))
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
                # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")
            df = pd.read_excel(io.BytesIO(file_data))

        elif blob.name.endswith(".csv"):
            # Lê o formato  csv e renomeia
            print(f"Arquivo: {blob.name} sendo precessado")
            csv_data = blob.download_as_text()

            try:
                # Tenta carregar o arquivo em um DataFrame
                df = pd.read_csv(io.StringIO(csv_data))
                renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)
            except Exception as e:
              # Captura qualquer erro e exibe a mensagem
                print(f"❌ Erro ao carregar o arquivo {blob.name} : {e}")
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")

        elif blob.name.endswith(".sql"):
            # Já foi processado por outro colab. Nada a fazer.
            print(f"Arquivo: {blob.name} sendo precessado")

            df = sql_para_dataframe(blob)

            renomeia(df,blob, source_bucket_name,source_folder,dest_bucket_name, dest_folder)

        elif blob.name == source_folder:
            print(f"🆗 Arquivo nulo encontrado: {blob.name}, ignorando processamento.")

        else:
            print(f"❌ Arquivo: {blob.name} não está em um formato válido")
            # Deletar o arquivo origem
            try:
                blob.delete()
                print(f"🆗 Arquivo original {blob.name} excluído com sucesso da pasta dados_brutos.")
            except Exception as e:
                print(f"❌ Erro ao excluir o arquivo {blob.name} : {e}")


# Execução das Funções de Tratamento e Envio para Bucket (dados_pre_processados)

In [ ]:
source_bucket_name = "" # Insira a Bucket de Origem dos arquivos
source_folder = "" # Insira a Pasta de Origem dos arquivos
dest_bucket_name = "" # Insira a Bucket de Destino dos arquivos
dest_folder = "" # Insira a pasta de Destino dos arquivos
processa_arquivos(source_bucket_name, source_folder, dest_bucket_name, dest_folder)

🆗 Arquivo nulo encontrado: dados_brutos/, ignorando processamento.


# Tratamento Final e envio para BigQuery

### Função para o Tratamento Geral dos DataFrames

In [ ]:
def tratamento(dataframe):
    """
    Função para tratar um DataFrame:
    - Remove colunas específicas.
    - Verifica e trata valores nulos, vazios e duplicados.
    - Trata valores string, aplicando formatação apropriada.
    - Ajusta os nomes das colunas para atender aos requisitos do BigQuery.
    """

    excecoes = ["de", "do", "da", "dos", "das"]

    # Remover colunas específicas
    try:
        dataframe = dataframe.drop(["mes referencia", "ano referencia"], axis=1)
        print("Êxito na exclusão das colunas.")
    except KeyError:
        print("Uma ou mais colunas não existem no DataFrame.")

    # Verificar valores nulos
    valores_nulos = dataframe.isna()
    contagem_nulos = valores_nulos.sum()
    if contagem_nulos.sum() > 0:
        print("Existem valores nulos no DataFrame:")
        print(contagem_nulos)
        print("\nDataFrame com valores nulos:\n", dataframe[valores_nulos.any(axis=1)].head())
    else:
        print("Não foram encontrados valores nulos no DataFrame.")

    # Verificar valores vazios
    valores_vazios = dataframe == ""
    contagem_vazios = valores_vazios.sum()
    if contagem_vazios.sum() > 0:
        print("Existem valores vazios no DataFrame:")
        print(contagem_vazios)
        print("\nDataFrame com valores vazios:\n", dataframe[valores_vazios.any(axis=1)].head())
    else:
        print("Não foram encontrados valores vazios no DataFrame.")

    # Remover duplicatas
    duplicatas = dataframe.duplicated()
    total_duplicatas = duplicatas.sum()
    if total_duplicatas > 0:
        print(f"Foram encontradas {total_duplicatas} linhas duplicadas no DataFrame.")
        dataframe = dataframe.drop_duplicates().reset_index(drop=True)
        print("As duplicatas foram removidas.")
    else:
        print("Não foram encontradas duplicatas no DataFrame.\n")

    # Tratar valores string
    for coluna in dataframe.columns:
        antes_strip = dataframe[coluna].copy()
        if dataframe[coluna].dtype == "object":
            dataframe.fillna("não informado", inplace=True)
            dataframe[coluna] = dataframe[coluna].str.strip().apply(
                lambda x: x.title() if isinstance(x, str) else x
            )
            dataframe[coluna] = dataframe[coluna].apply(
                lambda x: " ".join(
                    [
                        palavra.lower() if palavra.lower() in excecoes else palavra
                        for palavra in x.split()
                    ]
                )
                if isinstance(x, str)
                else x
            )
            if not antes_strip.equals(dataframe[coluna]):
                print(f"A coluna '{coluna}' foi tratada para remover espaços em branco extras e aplicar o título.")
            else:
                print(f"A coluna '{coluna}' não continha espaços em branco extras.")
        else:
            print(f"A coluna '{coluna}' não é do tipo string e não foi tratada.")

    # Ajustar nomes das colunas
    def ajustar_nome_coluna(coluna):

        coluna = ''.join(
            char for char in unicodedata.normalize('NFD', coluna)
            if unicodedata.category(char) != 'Mn'
        )
        coluna = coluna.strip().lower()
        coluna = re.sub(r"[^\w\s]", "", coluna)
        coluna = coluna.replace(" ", "_")
        return coluna[:300]

    colunas_originais = dataframe.columns.tolist()
    colunas_tratadas = [ajustar_nome_coluna(col) for col in colunas_originais]

    if colunas_tratadas != colunas_originais:
        print("\nNomes de colunas foram tratados:")
        print(f"Antes: {colunas_originais}")
        print(f"Depois: {colunas_tratadas}")
        dataframe.columns = colunas_tratadas

    print("\nInformações do DataFrame após o tratamento:")
    dataframe.info()
    print("\nPrévia do DataFrame tratado:")
    display(dataframe.head())

    return dataframe

## Função para Leitura do Arquivo CSV da pasta de Origem

In [ ]:
def read_csv_storage(arquivo):
    bucket_name = "" # Insira o nome da Bucket de onde os arquivos serão lidos
    blob_name = f"dados_pre_processados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    with blob.open("r") as f:
      arquivo = pd.read_csv(f)

    return arquivo

## Leitura e tratamento específicos para cada DataSet

### `ativos_clt`

In [ ]:
try:
  df_ativos_clt = read_csv_storage("ativos_clt")
except:
  print("Insira corretamente o nome do arquivo.")

In [ ]:
display(df_ativos_clt.head())
df_ativos_clt.info()

In [ ]:
df_ativos_clt["data admissao"] = df_ativos_clt["data admissao"].astype(str)
df_ativos_clt["data admissao"] = pd.to_datetime(df_ativos_clt["data admissao"],format="%m/%d/%Y")
df_ativos_clt["data nascimento"] = df_ativos_clt["data nascimento"].astype(str)
df_ativos_clt["data nascimento"] = pd.to_datetime(df_ativos_clt["data nascimento"],format="%m/%d/%Y")
df_ativos_clt = df_ativos_clt.rename(columns={"cpf/id" : "id_funcionario"})
df_ativos_clt = df_ativos_clt.rename(columns={"nivel" : "lider"})
df_ativos_clt.loc[df_ativos_clt["lider"].str.contains("-", na=False), "lider"] = "não"
df_ativos_clt.loc[df_ativos_clt["lider"].str.contains("liderança", na=False), "lider"] = "sim"
df_ativos_clt[["area","setor"]] = df_ativos_clt["area"].str.split(" - ", n = 1, expand = True)
df_ativos_clt["setor"] = df_ativos_clt["setor"].fillna(df_ativos_clt["area"])

In [ ]:
df_ativos_clt = tratamento(df_ativos_clt)

### `ativos_pj`

In [ ]:
try:
  df_ativos_pj = read_csv_storage("ativos_pj")
except:
  print("Insira corretamente o nome do arquivo.")

In [ ]:
df_ativos_pj.head()

In [ ]:
df_ativos_pj["data admissao"] = df_ativos_pj["data admissao"].astype(str)
df_ativos_pj["data admissao"] = pd.to_datetime(df_ativos_pj["data admissao"])
df_ativos_pj["data nascimento"] = df_ativos_pj["data nascimento"].astype(str)
df_ativos_pj["data nascimento"] = pd.to_datetime(df_ativos_pj["data nascimento"])
df_ativos_pj = df_ativos_pj.rename(columns={"cpf/id" : "id_funcionario"})
df_ativos_pj = df_ativos_pj.rename(columns={"nivel" : "lider"})
df_ativos_pj.loc[df_ativos_pj["lider"].str.contains("-", na=False), "lider"] = "não"
df_ativos_pj.loc[df_ativos_pj["lider"].str.contains("liderança", na=False), "lider"] = "sim"
df_ativos_pj["area"] = df_ativos_pj["area"].str.split(" - ", n = 1, expand = True)

In [ ]:
df_ativos_pj = tratamento(df_ativos_pj)

### `censo_diversidade`

In [ ]:
try:
  df_censo = read_csv_storage("censo_diversidade")
except:
  print("Insira corretamente o nome do arquivo.")

In [ ]:
display(df_censo.head())
df_censo.info()

In [ ]:
df_censo = df_censo.rename(
    columns={"De acordo com a Lei Geral de Proteção de Dados Pessoais, você está consentindo com a coleta e uso de dados pessoais sensíveis seus, tais como raça, identidade de gênero, orientação sexual e outros, qu" :"Consentimento da coleta e uso de dados pessoais",
             "True" : "Estou de acordo em responder",
             "False" : "Não estou de acordo em responder",
             "Qual sua idade?" : "idade",
             "Qual seu nome social? O nome social é o nome pelo qual uma pessoa é conhecida e reconhecida socialmente, que pode ser diferente do seu nome de registro civil." :"Nome social","Qual posição você ocupa?" : "Posição",
             "Coloque o nome do seu cargo":"Cargo",
             "Qual seu estado de nascimento?": "Estado de nascimento",
             "Qual estado você mora?" : "Estado que mora",
             "Em qual região de São Paulo você mora?" : "Região de São Paulo que mora",
             "Qual seu status de relacionamento afetivo?" : "Status de relacionamento afetivo",
             "Existe alguma restrição alimentar que devemos considerar ao planejar eventos ou reuniões? Se sim, justifique." :"Restrição alimentar",
             "Você tem alguma sensibilidade a algum alimento ou substância? Se sim, justifique." :"Sensibilidade a alimento ou substância",
             "Você tem algum problema de saúde/ alergias ou doença crônica que precisamos estar cientes? Se sim, justifique." :"Problema de saúde, alergias ou doença crônica",
             "Como você se define em termos de identidade de gênero?":"Identidade de gênero",
             "Como você se define em termos de sexo biologico":"Sexo biológico",
             "Como você se define em termos de orientação sexual?":"Orientação sexual",
             "Você se identifica como pessoa com deficiência?":"Identificação com deficiência",
             "Etnia / raça / cor: Como você se identifica?":"Etnia, raça e cor",
             "Qual o nível máximo da sua educação formal?" :"Nível máximo de educação formal",
             "Qual domínio com idiomas? Português":"Domínio com idioma Português","Qual domínio com idiomas? Inglês":"Domínio com idioma Inglês",
             "Qual domínio com idiomas? Espanhol" :"Domínio com idioma Espanhol",
    }
)

In [ ]:
df_censo[["Data de abertura", "Data de conclusão"]] = df_censo[
    ["Data de abertura", "Data de conclusão"]].apply(
    pd.to_datetime, errors="coerce"
)
df_censo.info()

In [ ]:
df_censo = tratamento(df_censo)

In [ ]:
df_censo['email'] = df_censo['email'].str.lower()

In [ ]:
df_censo = df_censo.replace({
    True: "Sim",
    False: "Não",
    "Amarelo(A)": "Amarelo(a)",
    "Negro(A)": "Negro(a)",
    "Branco(A)": "Branco(a)",
    "Pardo(A)": "Pardo(a)",
    "Indígena": "Indígena",
    "Male": "Masculino",
    "Female": "Feminino",
    "Bigender": "Bigênero",
    "Polygender": "Poligênero",
    "Genderfluid": "Gênero-fluido",
    "Non-Binary": "Não-binário",
    "Genderqueer": "Gênero queer"
})

# Exibir o DataFrame resultante
display(df_censo.head(3))

In [ ]:
display(df_censo.head(2))
df_censo.info()

### `ferias_clt`

In [ ]:
try:
  df_ferias_clt = read_csv_storage('ferias_clt')
except:
  print('Insira corretamente o nome do arquivo.')

In [ ]:
df_ferias_clt.head()

In [ ]:
df_ferias_clt = df_ferias_clt.drop(columns=["Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11"])
df_ferias_clt.head()

In [ ]:
df_ferias_clt["duracao"] = df_ferias_clt["duracao"].fillna(0)
df_ferias_clt["duracao"] = df_ferias_clt["duracao"].round(0)
df_ferias_clt["duracao"] = df_ferias_clt["duracao"].astype(int)

display(df_ferias_clt)

In [ ]:
df_ferias_clt = tratamento(df_ferias_clt)

In [ ]:
if "cpfid" in df_ferias_clt.columns:
    df_ferias_clt.rename(columns={"cpfid" : "id_funcionario"}, inplace=True)
    print("Coluna 'cpfid' alterada para 'id_funcionario'.")
else:
    print("Não existe coluna 'cpfid'.")

if "id_funcionario" in df_ferias_clt.columns:
    df_ferias_clt['id_funcionario'] = df_ferias_clt['id_funcionario'].apply(lambda x: x if isinstance(x, str) and '-' in x else 'Não Informado')
else:
    None

df_ferias_clt["data_inicio"] = pd.to_datetime(
    df_ferias_clt["data_inicio"], errors="coerce"
)
df_ferias_clt["data_termino"] = pd.to_datetime(
    df_ferias_clt["data_termino"], errors="coerce"
)
df_ferias_clt = df_ferias_clt.drop(index=258).reset_index(drop=True)

In [ ]:
df_ferias_clt.head()

### `ferias_pj`

In [ ]:
try:
  df_ferias_pj = read_csv_storage('ferias_pj')
except:
  print('Insira corretamente o nome do arquivo.')

In [ ]:
df_ferias_pj.head()

In [ ]:
df_ferias_pj = tratamento(df_ferias_pj)

In [ ]:
df_ferias_pj = df_ferias_pj.rename(columns={"quant_dias": "duracao"})

ordem_colunas = ["data_solicitacao", "id_funcionario", "nome", "area", "data_inicio", "data_termino", "duracao"]
display(df_ferias_pj.head())

In [ ]:
if "cpfid" in df_ferias_pj.columns:
    df_ferias_pj.rename(columns={"cpfid" : "id_funcionario"}, inplace=True)
    print("Coluna 'cpfid' alterada para 'id_funcionario'.")
else:
    print("Não existe coluna 'cpfid'.")

if "id_funcionario" in df_ferias_pj.columns:
    df_ferias_pj['id_funcionario'] = df_ferias_pj['id_funcionario'].apply(lambda x: x if isinstance(x, str) and '-' in x else 'nao informado')
else:
    None

df_ferias_pj["data_solicitacao"] = pd.to_datetime(
    df_ferias_pj["data_solicitacao"], errors="coerce"
)
df_ferias_pj["data_inicio"] = pd.to_datetime(
    df_ferias_pj["data_inicio"], errors="coerce"
)
df_ferias_pj["data_termino"] = pd.to_datetime(
    df_ferias_pj["data_termino"], errors="coerce"
)

In [ ]:
df_ferias_pj.info()

### `relacao_desligados`

In [ ]:
try:
  df_relacao_desligados = read_csv_storage("relacao_desligados")
except:
  print("Insira corretamente o nome do arquivo.")

In [ ]:
df_relacao_desligados.head()

,mes referencia,ano referencia,id_funcionario,nome,tipo_desligamento,data_demissao,cargo
0,junho,2024,870-26-5986,Gerard Durran,Dispensa sem Justa Causa,2024-10-20,diretor de criacao
1,abril,2024,866-26-3608,Jard Dugget,Dispensa sem Justa Causa,2024-02-29,executivo de contas
2,junho,2024,260-49-9909,Maure Ferguson,Pedido de Demissão,2024-07-21,diretor de criacao
3,julho,2024,330-13-2767,Marguerite Jupe,Dispensa sem Justa Causa,2024-09-21,coordenador de midia
4,junho,2024,190-45-1192,Olivier Hurche,Pedido de Demissão,2024-04-04,supervisor de planejamento


In [ ]:
df_relacao_desligados = tratamento(df_relacao_desligados)

### `vagas_rs`

In [ ]:
try:
  df_vagas_rs = read_csv_storage("vagas_rs")
except:
  print("Insira corretamente o nome do arquivo.")

In [ ]:
df_vagas_rs = tratamento(df_vagas_rs)

In [ ]:
df_vagas_rs["abertura_vaga"] = df_vagas_rs["abertura_vaga"].astype(str)
df_vagas_rs["abertura_vaga"] = pd.to_datetime(df_vagas_rs["abertura_vaga"], errors="coerce")
df_vagas_rs["fechamento_da_vaga"] = df_vagas_rs["fechamento_da_vaga"].astype(str)
df_vagas_rs["fechamento_da_vaga"] = pd.to_datetime(df_vagas_rs["fechamento_da_vaga"], errors="coerce")

df_vagas_rs = df_vagas_rs.rename(columns={
    "substituicao__nova_vaga":"motivo_vaga",
    "tipo_de_diversidade":"tipo_diversidade",
    "identidade_de_genero":"identidade_genero"
})

df_vagas_rs["vinculo"] = df_vagas_rs["vinculo"].replace({
    "Clt":"CLT",
    "Pj": "PJ"
})

df_vagas_rs["area"] = df_vagas_rs["area"].replace({
    "Bi":"BI"
})

In [ ]:
display(df_vagas_rs.head())
df_vagas_rs.info()

## Geração do Arquivo CSV, Download para a máquina e Envio para o Cloud Storage e BigQuery

### `ativos_clt`

In [ ]:
try:
    df_ativos_clt.to_csv("ativos_clt.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'ativos_clt.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("ativos_clt.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("ativos_clt")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.ativos_clt" # Substitua o nome do arquivo

try:
    to_gbq(df_ativos_clt, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `ativos_pj`

In [ ]:
try:
    df_ativos_pj.to_csv("ativos_pj.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'ativos_pj.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("ativos_pj.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("ativos_pj")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.ativos_pj" # Substitua o nome do arquivo

try:
    to_gbq(df_ativos_pj, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `censo_diversidade`

In [ ]:
try:
    df_censo.to_csv("censo_diversidade.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'censo_diversidade.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("censo_diversidade.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("censo_diversidade")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.censo_diversidade" # Substitua o nome do arquivo

try:
    to_gbq(df_censo, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `ferias_clt`

In [ ]:
try:
    df_ferias_clt.to_csv("ferias_clt.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'ferias_clt.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("ferias_clt.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("ferias_clt")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.ferias_clt" # Substitua o nome do arquivo

try:
    to_gbq(df_ferias_clt, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `ferias_pj`

In [ ]:
try:
    df_ferias_pj.to_csv("ferias_pj.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'ferias_pj.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("ferias_pj.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("ferias_pj")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.ferias_pj" # Substitua o nome do arquivo

try:
    to_gbq(df_ferias_pj, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `relacao_desligados`



In [ ]:
try:
    df_relacao_desligados.to_csv("relacao_desligados.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'relacao_desligados.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("relacao_desligados.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("relacao_desligados")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.relacao_desligados" # Substitua o nome do arquivo

try:
    to_gbq(df_relacao_desligados, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")

### `vagas_rs`

In [ ]:
try:
    df_vagas_rs.to_csv("vagas_rs.csv", index=False) # Substitua o nome do arquivo
    print("Arquivo CSV 'vagas_rs.csv' gerado com sucesso!\n") # Substitua o nome do arquivo
except Exception as e:
    print(f"Erro ao gerar o arquivo CSV: {e}\n")

try:
    files.download("vagas_rs.csv") # Substitua o nome do arquivo
    print("Download do arquivo CSV realizado com sucesso!\n")
except Exception as e:
    print(f"Erro ao fazer o download do arquivo CSV: {e}\n")

def write_csv_storage(arquivo):
    bucket_name = "" # Insira a Bucket de Destino dos arquivos
    blob_name = f"dados_tratados/{arquivo}.csv"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.upload_from_filename(f"{arquivo}.csv")

write_csv_storage("vagas_rs")  # Substitua o nome do arquivo
print("Arquivo enviado com sucesso para a a pasta 'dados_tratados'.\n")

table_id = "nome-do-projeto.nome-da-pasta.vagas_rs" # Substitua o nome do arquivo

try:
    to_gbq(df_vagas_rs, table_id, project_id=project_id, if_exists="replace")  # Substitua o nome do arquivo
    print("\n\nEnvio para o BigQuery realizado com sucesso!")
except Exception as e:
    print(f"\n\nErro ao enviar os dados para o BigQuery: {e}")